In [2]:
import sys 
import os

# Add the subdirectory path to sys.path
repo_path = "/Users/hajunhyeon/Documents/GitHub/KAIRI_MCD/monte-carlo-diffusion"
sys.path.append(repo_path)

path1= "/Users/hajunhyeon/Documents/GitHub/KAIRI_MCD/monte-carlo-diffusion/src" 
sys.path.append(path1)
# sys.path.append('C:/Users/user/Documents/GitHub/KAIRI_MCD/monte-carlo-diffusion') 
# %pip install C:/Users/user/Documents/GitHub/KAIRI_MCD/monte-carlo-diffusion 
%pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [22]:
%pip install equinox  
%pip install numpyro 


/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/pty.py:85: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement equinox.optimisers (from versions: none)
ERROR: No matching distribution found for equinox.optimisers
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install jax==0.4.25 jaxlib==0.4.25 
# %pip install numpy<2 

In [1]:
from typing import Callable, List

import equinox as eqx
import jax
import jax.numpy as jnp 
import optax 
from jax.random import PRNGKey, split
from jaxtyping import Array  # type: ignore 
from tqdm import tqdm 



class MCDNetResBlock(eqx.Module):
    """
    Residual MLP block from `Score-Based Diffusion meets Annealed Importance Sampling <https://arxiv.org/abs/2208.07698>`_.
    """

    d_t: int = eqx.static_field()
    d_h: int = eqx.static_field()
    norm: eqx.nn.LayerNorm
    t_emb_proj: eqx.nn.Linear
    h_proj: eqx.nn.Linear
    act: Callable[[Array], Array]
    deproj: eqx.nn.Linear

    def __init__(self, d_h, d_t, act=jax.nn.swish, *, key):
        super().__init__()
        self.d_t = d_t
        self.d_h = d_h
        self.act = act

        keys = split(key, 3)
        self.norm = eqx.nn.LayerNorm(d_h)
        self.t_emb_proj = eqx.nn.Linear(d_t, 2 * d_h, key=keys[1])
        self.h_proj = eqx.nn.Linear(d_h, 2 * d_h, key=keys[0])
        self.deproj = eqx.nn.Linear(2 * d_h, d_h, key=keys[2])

    def __call__(self, t_emb, h):
        h = self.norm(h)
        h = self.act(h)
        h = self.h_proj(h)
        h += self.t_emb_proj(t_emb)
        h = self.act(h)
        return self.deproj(h)


class MCDNet(eqx.Module):
    """
    Residual MLP network similar to the one used in the experiments in `Score-Based Diffusion meets Annealed Importance Sampling <https://arxiv.org/abs/2208.07698>`_.
    Parametrized as :math:`c_{\\theta}(t, x) + (1 + s_{\\theta}(t, x)) \\nabla \\log \\gamma_t(x)`.

    """

    x_dim: int = eqx.static_field()
    get_score_gamma_t: Callable[[Array, Array], Array]
    d_t: int = eqx.static_field()
    d_h: int = eqx.static_field()
    depth: int = eqx.static_field()
    act: Callable[[Array], Array]
    # Layers
    t_emb: eqx.nn.Linear
    x_emb: eqx.nn.Linear
    res_layers: List[MCDNetResBlock]
    final_layer: eqx.nn.Linear
    const_scale: Array
    """constant multiplying the :math:`c_{\\theta}(t, x)` network, initialized to zero.
    """
    score_scale: Array
    """constant multiplying the :math:`s_{\\theta}(t, x)` network, initialized to zero.
    """

    def __init__(
        self,
        x_dim: int,
        get_score_gamma_t: Callable[[Array, Array], Array],
        d_t: int = 4,
        d_h: int = 64,
        depth: int = 3,
        act: Callable[[Array], Array] = jax.nn.swish,
        *,
        key: PRNGKey
    ):
        """
        Initializes the network with the output set to :math:`\\gamma_t(x)`.

        Args:
            x_dim: dimensionality of `x`.
            get_score_gamma_t: :math:`\\gamma_t(x)`, which is used to initialize
                the network's output to the standard AIS backward kernel.
            d_t: dimensionality of `t` embedding.
            d_h: dimensionalixy of `x` embedding.
            depth: number of residual MLP blocks.
            act: activation function.
            key: PRNG key used to initialize layers.

        """
        super().__init__()
        self.x_dim = x_dim
        self.get_score_gamma_t = get_score_gamma_t
        self.d_t = d_t
        self.d_h = d_h
        self.depth = depth
        self.act = act

        # Embedding layers
        key_t, key_x, key_final, *keys_res = split(key, 3 + depth)
        self.t_emb = eqx.nn.Linear(1, d_t, key=key_t)
        self.x_emb = eqx.nn.Linear(x_dim, d_h, key=key_x)
        self.res_layers = [MCDNetResBlock(d_h, d_t, act, key=k) for k in keys_res]
        # Final layer
        self.final_layer = eqx.nn.Linear(d_h, 2 * x_dim, key=key_final)
        # Scaling for score term
        self.const_scale = jnp.array(0.0)
        self.score_scale = jnp.array(0.0)

    def __call__(self, t: Array, x: Array) -> Array:
        t_emb = self.t_emb(t[None])
        h = self.x_emb(x)
        for res_layer in self.res_layers:
            h = res_layer(t_emb, h)
        h = self.act(h)
        h = self.final_layer(h)
        # Scale terms by constants
        const = self.const_scale * h[: self.x_dim]
        score_factor = 1 + self.score_scale * h[: self.x_dim]
        # Combine with score
        score_t = self.get_score_gamma_t(t, x)
        return const + score_factor * score_t


In [3]:
import jax
import jax.numpy as jnp
import equinox as eqx 
import optax
from jax.random import PRNGKey, split
from typing import Callable, List
from mcd import UnadjustedLangevin
import numpyro.distributions as dist
import matplotlib.pyplot as plt
from tqdm import trange

/Users/hajunhyeon/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class MultivariateNormalDiag:
    def __init__(self, loc, scale_diag):
        self.loc = loc
        self.scale_diag = scale_diag

    def sample(self, key):
        """
        Generate a sample from the multivariate normal distribution.
        """
        return self.loc + self.scale_diag * jax.random.normal(key, self.loc.shape)

    def log_prob(self, x):
        """
        Compute the log probability of x under the multivariate normal distribution.
        """
        diff = (x - self.loc) / self.scale_diag
        log_det = jnp.sum(jnp.log(self.scale_diag))
        norm = -0.5 * self.loc.shape[0] * jnp.log(2 * jnp.pi)
        return norm - log_det - 0.5 * jnp.sum(diff**2) 
    
class GaussianMixture:
    def __init__(self, dim, n_components, stdev, key):
        self.dim = dim
        self.n_components = n_components
        self.stdev = stdev
        self.means = dist.Normal(jnp.zeros((dim,)), 3.0).sample(key, (n_components,))
    
    def _get_component_log_prob(self, x, mean):
        return dist.Normal(mean, self.stdev).log_prob(x)
    
    def get_log_prob(self, x):
        """Log probability of the Gaussian Mixture."""
        log_probs = jax.vmap(lambda mean: self._get_component_log_prob(x, mean))(self.means)
        return jnp.log(jnp.exp(log_probs).sum() / self.n_components)

def get_log_pi(t, x, get_log_gamma, pi_0, T):
    """Interpolated log probability for diffusion process."""
    return (1 - t / T) * pi_0.log_prob(x) + (t / T) * get_log_gamma(x)

def get_score_pi(t, x, get_log_gamma, pi_0, T):
    """Gradient of interpolated log probability."""
    return jax.grad(lambda x: get_log_pi(t, x, get_log_gamma, pi_0, T))(x)

In [12]:
# Define Residual MLP (MCDNet)
class ResBlock(eqx.Module):
    layer_norm: eqx.nn.LayerNorm
    t_linear: eqx.nn.Linear
    h_linear: eqx.nn.Linear
    final_linear: eqx.nn.Linear
    activation: Callable = eqx.static_field()

    def __init__(self, t_dim, h_dim, activation=jax.nn.swish, out_dim=None, *, key):
        key_t, key_h, key_final = split(key, 3)
        self.layer_norm = eqx.nn.LayerNorm(h_dim)
        self.t_linear = eqx.nn.Linear(t_dim, 2 * h_dim, key=key_t)
        self.h_linear = eqx.nn.Linear(h_dim, 2 * h_dim, key=key_h)
        if out_dim is None:
            out_dim = h_dim
        self.final_linear = eqx.nn.Linear(2 * h_dim, out_dim, key=key_final)
        self.activation = activation

    def __call__(self, t, h):
        t_emb = self.t_linear(t)
        h = self.layer_norm(h)
        h = self.activation(h)
        h = self.h_linear(h)
        h = t_emb + h
        h = self.activation(h)
        h = self.final_linear(h)
        return h


class Model(eqx.Module):
    t_scale: float = eqx.static_field()
    t_linear: eqx.nn.Linear
    x_linear: eqx.nn.Linear
    res_blocks: List[ResBlock]
    final_scale: jnp.ndarray 
    pi_0: MultivariateNormalDiag= eqx.static_field() 
    T: float = eqx.static_field() 
    get_log_gamma: Callable= eqx.static_field()

    def __init__(
        self, x_dim, t_dim, h_dim, depth, t_scale, pi_0, T, get_log_gamma, activation=jax.nn.swish, *, key
    ):
        self.t_scale = t_scale 
        self.pi_0= pi_0 
        self.T= T 
        self.get_log_gamma= get_log_gamma 

        keys = split(key, 4)
        self.t_linear = eqx.nn.Linear(1, t_dim, key=keys[0])
        self.x_linear = eqx.nn.Linear(x_dim, h_dim, key=keys[1]) 

        res_keys = split(keys[2], depth)
        self.res_blocks = []
        for i in range(depth - 1):
            self.res_blocks.append(ResBlock(t_dim, h_dim, activation=activation, key=res_keys[i]))
        self.res_blocks.append(ResBlock(t_dim, h_dim, activation=activation, out_dim=x_dim, key=res_keys[-1]))
        
        self.final_scale = jnp.zeros(1)

    def __call__(self, t, x):
        t_emb = self.t_linear(jnp.atleast_1d(t) * self.t_scale)
        h = self.x_linear(x)
        for res_block in self.res_blocks:
            h = res_block(t_emb, h)
        return self.final_scale * h + get_score_pi(t, x, self.get_log_gamma, self.pi_0, self.T)

In [13]:
def run_experiment(d, n_timesteps, description):
    key = PRNGKey(87)
    T = n_timesteps
    
    # Initialize Gaussian Mixture
    n_components = 8
    stdev = 1.0
    key, subkey = split(key)
    gaussian_mixture = GaussianMixture(d, n_components, stdev, subkey)
    
    # Initialize prior distribution
    scale = 3.0
    pi_0 = MultivariateNormalDiag(loc=jnp.zeros(d), scale_diag=scale * jnp.ones(d))
    
    # Initialize ULA
    ula = UnadjustedLangevin(
        pi_0, 
        get_log_gamma= gaussian_mixture.get_log_prob, 
        get_score_gamma_t= lambda t, x: get_score_pi(t, x, gaussian_mixture.get_log_prob, pi_0, T),
        n_timesteps= n_timesteps,
        T=T
    )

    # Initialize model
    key, subkey = split(key)
    t_dim = 4
    h_dim = 128
    depth = 3
    model = Model(
        x_dim=d, 
        t_dim= t_dim, 
        h_dim= h_dim, 
        depth=depth, 
        t_scale= ula.delta, 
        pi_0=pi_0, 
        T=T, 
        get_log_gamma= gaussian_mixture.get_log_prob, 
        key=subkey
    )

    # Training setup
    lr = 5e-3
    optim = optax.adam(lr)
    opt_state = optim.init(eqx.filter(model, eqx.is_inexact_array))
    batch_size = 512

    @eqx.filter_jit
    def train_step(model, key, opt_state, batch_size):
        keys = split(key, batch_size)
        get_loss = lambda model, keys: jax.vmap(ula.get_loss, (None, 0))(model, keys).sum()
        loss, grads = eqx.filter_value_and_grad(get_loss)(model, keys)
        updates, opt_state = optim.update(grads, opt_state)
        model = eqx.apply_updates(model, updates)
        return loss, model, opt_state

    # Training loop
    n_steps = 200
    losses = []
    with trange(n_steps, desc=description) as pbar:
        for _ in pbar:
            key, subkey = split(key)
            loss, model, opt_state = train_step(model, subkey, opt_state, batch_size)
            losses.append(loss.item())
            pbar.set_postfix(loss=losses[-1])

    # Evaluate
    key, subkey = split(key)
    n_samples = 40_000
    xs_ais, log_ws_ais = jax.vmap(ula.get_trajectory_ais)(split(subkey, n_samples))
    xs_mcd, log_ws_mcd = jax.vmap(ula.get_trajectory_mcd, (None, 0))(model, split(subkey, n_samples))

    def compute_log_Z(log_weights):
        return jnp.log(jnp.mean(jnp.exp(log_weights)))

    log_Z_ais = compute_log_Z(log_ws_ais)
    log_Z_mcd = compute_log_Z(log_ws_mcd)

    return log_Z_ais, log_Z_mcd

if __name__ == "__main__":
    # Run experiment with dimension 20
    d = 20
    log_Z_ais, log_Z_mcd = run_experiment(d, 64, "Gaussian Mixture (Steps=64)")
    print(f"Gaussian Mixture (Steps=64): AIS Log Z = {log_Z_ais}, MCD Log Z = {log_Z_mcd}")

/var/folders/9y/kcw0q0811sv1mcwy1bs2b5z80000gn/T/ipykernel_23949/3393701735.py:29: UserWarning: A JAX array is being set as static! This can result in unexpected behavior and is usually a mistake to do.
  model = Model(
Gaussian Mixture (Steps=64):  50%|████▉     | 99/200 [00:41<00:41,  2.46it/s, loss=2.88e+5]